In [1]:
from neuprint import Client
 
import pandas as pd
from neuprint import fetch_adjacencies, NeuronCriteria as NC,merge_neuron_properties  

In [2]:
pd.__version__

'1.3.4'

In [3]:
#replace neuprint access token#
#see: https://neuprint.janelia.org/account #
user_token = ''


c = Client('https://neuprint.janelia.org/', dataset='hemibrain:v1.2.1',token= user_token)   

In [4]:
T2_IDs = [1963079392,
2213059115,
1995123997,
2025153433,
2118935033,
2181009844,
1994792005,
5812993999,
2088574169,
1994791924,
2118940139,
2149296373,
1901682750,
1840282178,
1995802079,
2026163864,
2182019184,
2150648466,
2026163864,
1901682750,
1995802079,
1840282178]
 

In [5]:
criteria = NC(bodyId=T2_IDs)
 
neuron_df1, conn_df1 = fetch_adjacencies(criteria, 
                                         None, 
                                         properties=['type', 'instance','status'],
                                         include_nonprimary=False)

conn_df = merge_neuron_properties(neuron_df1, conn_df1, ['type', 'instance','status'])
conn_df = conn_df[conn_df["status_post"]=='Traced']  # 'Traced' target neurons only

conn_df

,bodyId_pre,bodyId_post,roi,weight,type_pre,instance_pre,status_pre,type_post,instance_post,status_post
0,1840282178,1405106221,LO(R),3,None,None,Traced,None,None,Traced
1,1840282178,1498574596,LO(R),3,None,None,Traced,LC4,LC4,Traced
2,1840282178,1499912869,LO(R),1,None,None,Traced,None,None,Traced
3,1840282178,1528914548,LO(R),4,None,None,Traced,None,None,Traced
4,1840282178,1560285487,LO(R),4,None,None,Traced,None,(LWF1)_R,Traced
...,...,...,...,...,...,...,...,...,...,...
1181,5812993999,5812998041,LO(R),3,None,None,Traced,Li15,Li15_R,Traced
1182,5812993999,5813000482,LO(R),2,None,None,Traced,None,None,Traced
1183,5812993999,5813025000,LO(R),2,None,None,Traced,None,(PVF4)_R,Traced
1184,5812993999,5813033901,LO(R),1,None,None,Traced,None,None,Traced


In [7]:
connection_table = conn_df.groupby(by=['bodyId_pre','type_post'], as_index=False, dropna=False).sum()
connection_table = connection_table.pivot(index='bodyId_pre',columns='type_post', values ='weight').fillna(0)
connection_table['All'] = connection_table.sum(axis=1)
new_columns = [ x for x in connection_table.columns if x in ['LC4','LC6','LC9','LC10','LC11','LC12','LC13',
                                                            'LC15','LC16','LC17','LC18','LC20','LC21','LC22'
                                                             'LC24','LC25','LC26','LPLC1','LPLC2','LPLC4',
                                                             'All']]
connection_table=connection_table[new_columns]
connection_table.loc['Total',:]=connection_table.sum(axis=0)
connection_table.sort_values(by='Total',axis=1,ascending=False,inplace=True)


output_directory = './'
connection_table.to_csv(output_directory+'T2_to_LC.csv')  